In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

# **Model Bert**

In [8]:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

# Load tokenizer dan model yang sudah dilatih
model_path = "/content/drive/MyDrive/bert_intent_model"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# Inisialisasi pipeline untuk klasifikasi teks
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)


Device set to use cpu


In [9]:
intent_labels = {
    "kata-kasar": 0,
    "laporan-kekerasan": 1,
    "psikologi": 2,
    "data-umum": 3,
    "jumlah-kdrt": 4
}
# Balik: angka ke label
reverse_labels = {v: k for k, v in intent_labels.items()}


In [11]:

test_texts = [
    "aku ingin melapor kekerasan sekolah",
]

predictions = classifier(test_texts)

for text, pred in zip(test_texts, predictions):
    label_index = int(pred["label"].split("_")[1])
    intent = reverse_labels[label_index]
    print(f"Teks: {text}\nPrediksi Intent: {intent}\n")

Teks: aku ingin melapor kekerasan sekolah
Prediksi Intent: laporan-kekerasan



# **Model t5**

In [4]:
!pip install duckduckgo-search transformers
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00


In [14]:
import os

model_path = "/content/drive/MyDrive/T5_Checkpoints"
print(os.listdir(model_path))


['t5-tokenizer-50percent.zip', 't5-squad2-checkpoint.zip', 'Salinan t5-tokenizer-50percent.zip']


In [15]:
import zipfile
import os

zip_paths = [
    "/content/drive/MyDrive/T5_Checkpoints/t5-tokenizer-50percent.zip",
    "/content/drive/MyDrive/T5_Checkpoints/t5-squad2-checkpoint.zip"
]

for zip_path in zip_paths:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        extract_folder = zip_path.replace(".zip", "")
        zip_ref.extractall(extract_folder)
        print(f"✅ Berhasil ekstrak: {zip_path} ke {extract_folder}")


✅ Berhasil ekstrak: /content/drive/MyDrive/T5_Checkpoints/t5-tokenizer-50percent.zip ke /content/drive/MyDrive/T5_Checkpoints/t5-tokenizer-50percent
✅ Berhasil ekstrak: /content/drive/MyDrive/T5_Checkpoints/t5-squad2-checkpoint.zip ke /content/drive/MyDrive/T5_Checkpoints/t5-squad2-checkpoint


In [17]:
from duckduckgo_search import DDGS
from deep_translator import GoogleTranslator
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer_path = "/content/drive/MyDrive/T5_Checkpoints/t5-tokenizer-50percent"
model_path = "/content/drive/MyDrive/T5_Checkpoints/t5-squad2-checkpoint"

tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)



def search_context(query):
    """Cari informasi dari DuckDuckGo, ambil beberapa hasil, dan terjemahkan ke bahasa Inggris."""
    context_list = []

    try:
        with DDGS() as ddgs:
            results = ddgs.text(query, max_results=5)  # Ambil 3 hasil pencarian

        for r in results:
            if "body" in r and r["body"]:  # Pastikan ada isi
                context_list.append(r["body"])

        if not context_list:
            return "Context not found."

        # Gabungkan hasil menjadi satu teks
        context = " ".join(context_list)

        # Terjemahkan ke bahasa Inggris jika perlu
        translator = GoogleTranslator(source="auto", target="en")
        translated_context = translator.translate(context)

        return translated_context

    except Exception as e:
        return f"Error: {str(e)}"

def generate_answer(question, context):
    """Jawab pertanyaan dengan model T5"""
    if "Error" in context or "Context not found" in context:
        return "Sorry, I couldn't find relevant information."

    input_text = f"question: {question}  context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate jawaban
    output_ids = model.generate(input_ids, max_length=128)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Contoh pertanyaan dan pencarian konteks
question = "im so confused"

context = search_context(question)

# Coba jawab pertanyaan
answer = generate_answer(question, context)
print("Translated Context:", context)
print("Jawaban:", answer)

predictions = classifier(question)

for text, pred in zip(test_texts, predictions):
    label_index = int(pred["label"].split("_")[1])
    intent = reverse_labels[label_index]
    print(f"Teks: {text}\nPrediksi Intent: {intent}\n")

Translated Context: Learn why confusion arises and how to overcome it with 8 pointers that will help you stay in the present, feel the fear, and trust life. Find out how to stop thinking and get into stillness, and how to embrace the mystery of life. Learn how to distinguish between your true nature of clarity and the confusing thoughts that create the illusion of confusion. Discover how to access your clarity and make decisions based on your deepest knowing, not on your insecure or fearful thinking. Since 2006, I've helped more than a few people with that feeling of "lost and confused," helping them to shift from confused, to clarity. Lost and Confused: Help is Here. First, when you're feeling lost and confused, do this: don't do anything. Not doing something when you feel lost and confused is a healthy action. Most of us go back and ... Use deep breathing to trigger your body's relaxation response. Slow, guided breathing improves your mood and focus and reduces anxiety. When you feel

In [18]:

predictions = classifier(question)

for text, pred in zip(test_texts, predictions):
    label_index = int(pred["label"].split("_")[1])
    intent = reverse_labels[label_index]
    print(f"Teks: {text}\nPrediksi Intent: {intent}\n")

Teks: aku ingin melapor kekerasan sekolah
Prediksi Intent: data-umum



# **Finish**

In [21]:
from duckduckgo_search import DDGS
from deep_translator import GoogleTranslator
from transformers import T5Tokenizer, T5ForConditionalGeneration, BertTokenizer, BertForSequenceClassification, pipeline

# ========== Load Model T5 ==========
tokenizer_t5 = T5Tokenizer.from_pretrained("/content/drive/MyDrive/T5_Checkpoints/t5-tokenizer-50percent")
model_t5 = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/T5_Checkpoints/t5-squad2-checkpoint")

# ========== Load Model BERT Intent ==========
bert_model_path = "/content/drive/MyDrive/bert_intent_model"
tokenizer_bert = BertTokenizer.from_pretrained(bert_model_path)
model_bert = BertForSequenceClassification.from_pretrained(bert_model_path)
classifier = pipeline("text-classification", model=model_bert, tokenizer=tokenizer_bert)

# ========== Label Mapping ==========
intent_labels = {
    "kata-kasar": 0,
    "laporan-kekerasan": 1,
    "psikologi": 2,
    "data-umum": 3,
    "jumlah-kdrt": 4
}
reverse_labels = {v: k for k, v in intent_labels.items()}

# ========== Fungsi Cari Konteks ==========
def search_context(query):
    """Cari informasi via DuckDuckGo dan terjemahkan hasilnya ke Inggris."""
    context_list = []
    try:
        with DDGS() as ddgs:
            results = ddgs.text(query, max_results=5)

        for r in results:
            if "body" in r and r["body"]:
                context_list.append(r["body"])

        if not context_list:
            return "Context not found."

        combined_context = " ".join(context_list)

        # Terjemahkan hasil konteks ke Inggris
        translator = GoogleTranslator(source="auto", target="en")
        translated_context = translator.translate(combined_context)

        return translated_context

    except Exception as e:
        return f"Error: {str(e)}"

# ========== Fungsi Jawab Pertanyaan ==========
def generate_answer(question, context):
    """Jawab pertanyaan dengan model T5. Terjemahkan pertanyaan ke Inggris, hasilnya ke Indonesia."""
    if "Error" in context or "Context not found" in context:
        return "Maaf, aku tidak bisa menemukan informasi yang relevan."

    # Terjemahkan pertanyaan ke Inggris
    translated_question = GoogleTranslator(source="auto", target="en").translate(question)

    # Format input untuk T5
    input_text = f"question: {translated_question}  context: {context}"
    input_ids = tokenizer_t5(input_text, return_tensors="pt").input_ids

    # Generate jawaban
    output_ids = model_t5.generate(input_ids, max_length=128)
    english_answer = tokenizer_t5.decode(output_ids[0], skip_special_tokens=True)

    # Terjemahkan jawaban ke Bahasa Indonesia
    indonesian_answer = GoogleTranslator(source="en", target="id").translate(english_answer)
    return indonesian_answer

# ========== Fungsi Prediksi Intent (BERT) ==========
def predict_intent(texts):
    """Prediksi intent menggunakan model BERT (tanpa translate)."""
    predictions = classifier(texts)
    results = []
    for text, pred in zip(texts, predictions):
        label_index = int(pred["label"].split("_")[1])
        intent = reverse_labels[label_index]
        results.append((text, intent))
    return results

# ========== CONTOH PENGGUNAAN ==========

user_question = "aku bingung banget sama hidupku"

# 1. Cari konteks dari internet
context = search_context(user_question)

# 2. Jawab pakai T5 (terjemahin bolak-balik)
answer = generate_answer(user_question, context)

# 3. Prediksi intent pakai BERT (tanpa translate)
intent_results = predict_intent([user_question])

# ========== Output ==========
print("📚 Konteks (terjemahan Inggris):", context)
print("🧠 Jawaban (dalam Bahasa Indonesia):", answer)
for teks, intent in intent_results:
    print(f"💬 Teks: {teks}\n📌 Prediksi Intent (BERT): {intent}")


Device set to use cpu


📚 Konteks (terjemahan Inggris): I live, but I feel confused with my own life. Confused, where do you take this life. It feels like being swayed in a busy world. Often this confusion makes me feel ... Have you ever been confused about your life? I used to when I was young, I was confused, I didn't know what I had to say with my life. Then at that time I did not have mentors who could give motivation and guide me to find what I like. It doesn't taste really good. It feels like you get lost and doubt the same as my future will be. Speaking of life, sometimes I'm confused by that one 'thing'. Everyone has his own way of living his life, but I'm confused by my way. Feeling confused about the purpose of life is common and experienced by many people. However, it is important to overcome this feeling with positive steps. Remember that you have the ability to live a life full of meaning and satisfaction. Try to make the feeling of confusion as an opportunity to ... But there are times when we a